---
# **LAB 4 - CUDA in Python**
---

# ▶️ CUDA setup

In [17]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [18]:
!nvidia-smi

Tue Mar 22 09:29:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    75W / 149W |   2348MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

[GPU Compute Capability](https://developer.nvidia.com/cuda-gpus)

In [19]:
from numba import cuda

# getting device information
cuda.detect();

Found 1 CUDA devices
id 0            b'Tesla K80'                              [SUPPORTED]
                      compute capability: 3.7
                           pci device id: 4
                              pci bus id: 0
Summary:
	1/1 devices are supported


In [20]:
from numba import cuda

attribs = ['COMPUTE_CAPABILITY', 'MULTIPROCESSOR_COUNT', 'MAX_REGISTERS_PER_BLOCK', 
         'MAX_SHARED_MEMORY_PER_BLOCK', ]
device = cuda.get_current_device()
#attribs = [s for s in dir(device) if s.isupper()]        # all attribs
for attr in attribs:
    print(attr, '=', getattr(device, attr))

COMPUTE_CAPABILITY = (3, 7)
MULTIPROCESSOR_COUNT = 13
MAX_REGISTERS_PER_BLOCK = 65536
MAX_SHARED_MEMORY_PER_BLOCK = 49152


In [21]:
#@title Bash setup
%%writefile /root/.bashrc

# If not running interactively, don't do anything
[ -z "$PS1" ] && return

# don't put duplicate lines in the history. See bash(1) for more options
# ... or force ignoredups and ignorespace
HISTCONTROL=ignoredups:ignorespace

# append to the history file, don't overwrite it
shopt -s histappend

# for setting history length see HISTSIZE and HISTFILESIZE in bash(1)
HISTSIZE=10000
HISTFILESIZE=20000

# check the window size after each command and, if necessary,
# update the values of LINES and COLUMNS.
shopt -s checkwinsize

# make less more friendly for non-text input files, see lesspipe(1)
[ -x /usr/bin/lesspipe ] && eval "$(SHELL=/bin/sh lesspipe)"

PS1='\[\033[01;34m\]\w\[\033[00m\]\$ '

# enable color support of ls and also add handy aliases
if [ -x /usr/bin/dircolors ]; then
    test -r ~/.dircolors && eval "$(dircolors -b ~/.dircolors)" || eval "$(dircolors -b)"
    alias ls='ls --color=auto'
    #alias dir='dir --color=auto'
    #alias vdir='vdir --color=auto'

    alias grep='grep --color=auto'
    alias fgrep='fgrep --color=auto'
    alias egrep='egrep --color=auto'
fi

# some more ls aliases
alias ll='ls -lF'
alias la='ls -A'
alias l='ls -CF'

# path setup
export PATH="./:/usr/local/cuda/bin:$PATH"

!source /root/.bashrc

Overwriting /root/.bashrc


Clone GPUcomputing site on github...

In [22]:
!git clone https://github.com/giulianogrossi/GPUcomputing.git

fatal: destination path 'GPUcomputing' already exists and is not an empty directory.


# 🐍 Add kernel

In [ ]:
from numba import cuda
import numpy as np

In [ ]:
# CUDA kernels written with `@cuda.jit` do not return values
@cuda.jit
def add_kernel(x, y, z):
  """Perform vector sum z = x * y
  """
    
  # The actual values of the following CUDA-provided variables for thread and block indices,
  # like function parameters, are not known until the kernel is launched.
  
  # This calculation gives a unique thread index within the entire grid (see the slides above for more)
  idx = cuda.grid(1)      # 1 = one dimensional thread grid, returns a single value.
                          # This Numba-provided convenience function is equivalent to
                          # `cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x`

  # This thread will do the work on the data element with the same index as its own
  # unique index within the grid.
  z[idx] = x[idx] + y[idx]

Define the kernel parameters...

In [ ]:
# Because of how we wrote the kernel above, we need to have a 1 thread to one data element mapping,
# therefore we define the number of threads in the grid (128*32) to equal n (4096).
threads_per_block = 128
blocks_per_grid = 32

Generate data and device setup/transfert...

In [ ]:
# populate data
n = 4096
x = np.arange(n).astype(np.int32) # [0...4095] on the host
y = np.ones_like(x)               # [1...1] on the host

d_x = cuda.to_device(x) # Copy of x on the device
d_y = cuda.to_device(y) # Copy of y on the device
d_out = cuda.device_array_like(d_x) # Like np.array_like, but for device arrays


launch the kernel...

In [ ]:
%%time
add_kernel[blocks_per_grid, threads_per_block](d_x, d_y, d_out)
cuda.synchronize()
print(d_out.copy_to_host()) # Should be [1...4096]

[   1    2    3 ... 4094 4095 4096]
CPU times: user 267 ms, sys: 136 ms, total: 402 ms
Wall time: 523 ms


# 🐍 Mat multiplication kernel

A tutorial that explains NumPy axes: [Numpy axes](https://www.sharpsightlabs.com/blog/numpy-axes-explained/)

In [23]:
from numba import cuda
import numpy as np

In [30]:
@cuda.jit
def matmul_gpu(A, B, C):
  """
    Perform square matrix multiplication of C = A * B
  """
  row, col = cuda.grid(2)
  if row < C.shape[0] and col < C.shape[1]:
    tmp = 0.
    for k in range(A.shape[1]):
      tmp += A[row, k] * B[k, col]
    C[row, col] = tmp

# matrix multiplication using the cpu and C-like programming style (** strongly discouraged **)
def matmul_cpu(A, B):
  """
  Perform square matrix multiplication of C = A * B
  """
  C = np.zeros((A.shape[1],B.shape[1]))
  for i in range(A.shape[1]):
    for j in range(B.shape[0]):
      tmp = 0.                            
      for k in range(A.shape[1]):
        tmp += A[i, k] * B[k, j]   # multiply elements in row i of A and column j of B and add to temp
      C[i, j] = tmp    

In [50]:
# generate random vals
np.random.seed(42)
SIZE = 5000
A = np.ones((SIZE,SIZE)).astype('float32')  # mat 1
B = np.ones((SIZE,SIZE)).astype('float32')  # mat 2
#C = np.zeros((SIZE,SIZE)).astype('float32')                       # mat where we store answer 

d_A = cuda.to_device(A) # Copy of A on the device
d_B = cuda.to_device(B) # Copy of B on the device
d_C = cuda.device_array_like(A) # malloc on the device

# data type
d_A.dtype

dtype('float32')

In [54]:
!nvidia-smi

Tue Mar 22 09:45:08 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0    74W / 149W |   1108MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [55]:
print(C)

[[231. 231. 231. ...   0.   0.   0.]
 [231. 231. 231. ...   0.   0.   0.]
 [231. 231. 231. ...   0.   0.   0.]
 ...
 [  0.   0.   0. ... 239. 239. 239.]
 [  0.   0.   0. ... 239. 239. 239.]
 [  0.   0.   0. ... 239. 239. 239.]]


Kernel parameters...

In [56]:
block = (16, 16)  # each block will contain 16x16 threads, typically 128 - 512 threads/block
grid_x = int(np.ceil(C.shape[0] / block[0]))
grid_y = int(np.ceil(C.shape[1] / block[1]))
grid = (grid_x, grid_y)  # we calculate the gridsize (number of blocks) from array
print(grid)
print(f"The kernel will be executed up to element {block[0]*grid_x}")

(625, 625)
The kernel will be executed up to element 10000


In [57]:
# execution of the kernel
start = time.time()
matmul_gpu[grid, block](d_A, d_B, d_C)
# host and device sync
cuda.synchronize()
end = time.time()
print("Elapsed (sec) = %s" % (end - start))


Elapsed (sec) = 14.931994676589966


In [ ]:
C = d_C.copy_to_host()
print(C)

[[1000. 1000. 1000. ... 1000. 1000. 1000.]
 [1000. 1000. 1000. ... 1000. 1000. 1000.]
 [1000. 1000. 1000. ... 1000. 1000. 1000.]
 ...
 [1000. 1000. 1000. ... 1000. 1000. 1000.]
 [1000. 1000. 1000. ... 1000. 1000. 1000.]
 [1000. 1000. 1000. ... 1000. 1000. 1000.]]


In [ ]:
%%time
# execution of the python function in C-like programming style (strongly discouraged)
D = matmul_cpu(A, B)

In [60]:
# using numpy function
start = time.time()
C = np.dot(A, B)
end = time.time()
print("Elapsed (sec) = %s" % (end - start))

Elapsed (sec) = 3.548175096511841


In [38]:
print(C)

[[10000. 10000. 10000. ... 10000. 10000. 10000.]
 [10000. 10000. 10000. ... 10000. 10000. 10000.]
 [10000. 10000. 10000. ... 10000. 10000. 10000.]
 ...
 [10000. 10000. 10000. ... 10000. 10000. 10000.]
 [10000. 10000. 10000. ... 10000. 10000. 10000.]
 [10000. 10000. 10000. ... 10000. 10000. 10000.]]


# 🐍 Histogram

In [1]:
from numba import cuda
import numpy as np

## 🔴 TODO

In [2]:
@cuda.jit
def BMP_hist(I, hist):
  '''
  Increment bin counts in hist
    params:
      - I (uint8)  : HxWx3 matrix
      - hist (int) : 256x3 matrix
  '''

  i, j = cuda.grid(2)  
  R = I[i,j,0]
  G = I[i,j,1]
  B = I[i,j,2]
  cuda.atomic.add(hist[0], R, 1)
  cuda.atomic.add(hist[1], G, 1)
  cuda.atomic.add(hist[2], B, 1)

## Gestione immagine e kernel...

In [3]:
from PIL import Image

#image.save('beach1.bmp')
img = Image.open('GPUcomputing/images/dog.bmp')
img_rgb = img.convert('RGB')
img_rgb

FileNotFoundError: ignored

In [ ]:
# converto to numpy (host)
I = np.asarray(img)
print(f"Image size W x H x ch = {I.shape}")

# device data setup
d_I = cuda.to_device(I)
H = np.zeros((3,256)).astype(np.float32)
d_H = cuda.to_device(H)

Kernel parameters...

In [ ]:
block = (16, 16)  # each block will contain 16x16 threads, typically 128 - 512 threads/block
grid_x = int(np.ceil(I.shape[0] / block[0]))
grid_y = int(np.ceil(I.shape[1] / block[1]))
grid = (grid_x, grid_y)  # we calculate the gridsize (number of blocks) from array
print(grid)
print(f"The kernel will be executed up to element {block[0]*grid_x}")

In [ ]:
# kernel launch

BMP_hist[grid, block](d_I, d_H)
hist = d_H.copy_to_host()
print(hist.shape)

In [ ]:
import matplotlib.pyplot as plt

plt.bar(np.arange(256),hist[0])
plt.title('Histogram (R)')
plt.show()
plt.bar(np.arange(256),hist[1])
plt.title('Histogram (G)')
plt.show()
plt.bar(np.arange(256),hist[2])
plt.title('Histogram (B)')
plt.show()

# 🔴 TODO

Calcolare il prodotto di matrici MQDB con kernel CUDA in Python

In [61]:
from numba import cuda
import numpy as np
import time

def MQDB_gen(n, k, values=None):
  """
  Generate a random MQDB of size n with k (random sized) blocks
    params:
      n (int)                : mat size
      k (int or list)        : the number of blocks or blocks dims (list)
      values (string or num) : the string 'rand' for random values, 
                               a scalar for constant values everywere,
                               'None' for no assignement        
  """
  # blocks dims
  if isinstance(k, (list, np.ndarray)):
    blkdims = k  # list of already defined dims
  else:          # generate new dims
    bins = np.concatenate((np.ones(n), np.zeros(n*(k-1))))
    np.random.shuffle(bins)
    blkdims = np.sum(bins.reshape(k,n),axis=1).astype('int')
  print(f"Mat size n = {n}  --  num blocks k = {blkdims.shape[0]}")
  print('Blocks dims = ', blkdims)
  
  A = np.zeros((n,n))
  # filling of blocks 
  if values is not None:
    end = 0
    for b in blkdims:
      start = end
      end = end + b
      if values == 'rand':
        A[start:end,start:end] = np.random.rand(b,b)
      else:
        A[start:end,start:end] = values*np.ones(b,b)
      
  # return final data
  return A, blkdims

@cuda.jit
def matmul_MQDB(A, B, C, blkdim, blksum):
  """
  Perform MQDB matrix multiplication C = A * B
  params:
      A,B,C (MQDB) : omogeneous MQDB types 
      blkdim (int) : the current block dim
      blksum (int) : sum of the previous blocks sizes    
  """
  row, col = cuda.grid(2)
  if row < blkdim and col < blkdim:
    tmp = 0.
    for k in range(blkdim):
      tmp += A[row+blksum, k+blksum] * B[k+blksum, col+blksum]
    C[row+blksum, col+blksum] = tmp


In [62]:
# generate random blocks and data
n = 10000
k = 4
A, blkdims = MQDB_gen(n, k, values=1)
B, _ = MQDB_gen(n, blkdims, values=1)
C, _ = MQDB_gen(n, blkdims)

# device mem
d_A = cuda.to_device(A) # Copy of A on the device
d_B = cuda.to_device(B) # Copy of B on the device
d_C = cuda.device_array_like(A) # malloc on the device

Mat size n = 10000  --  num blocks k = 4
Blocks dims =  [2551 2479 2460 2510]
Mat size n = 10000  --  num blocks k = 4
Blocks dims =  [2551 2479 2460 2510]
Mat size n = 10000  --  num blocks k = 4
Blocks dims =  [2551 2479 2460 2510]


In [63]:
# run the kernels

# warm up 
matmul_MQDB[grid, block](d_A, d_B, d_C, blkdims[0], 0)

start = time.time()
block = (16, 16) # block size
for i in range(k):
  grid_x = int(np.ceil(blkdims[i]/block[0]))
  grid_y = int(np.ceil(blkdims[i]/block[1]))
  grid = (grid_x, grid_y)  
  sum_blks = 0 if i==0 else np.sum(blkdims[0:i])
  # kernel launch
  matmul_MQDB[grid, block](d_A, d_B, d_C, blkdims[i], sum_blks)

# host and device sync
cuda.synchronize()
end = time.time()
print("Elapsed (sec) = %s" % (end - start))


Elapsed (sec) = 9.475791454315186


In [64]:
# print results
print('results: matrx C')
import sys
C = d_C.copy_to_host() # copy to host from device
print(C)

results: matrx C
[[2551. 2551. 2551. ...    0.    0.    0.]
 [2551. 2551. 2551. ...    0.    0.    0.]
 [2551. 2551. 2551. ...    0.    0.    0.]
 ...
 [   0.    0.    0. ... 2510. 2510. 2510.]
 [   0.    0.    0. ... 2510. 2510. 2510.]
 [   0.    0.    0. ... 2510. 2510. 2510.]]


In [43]:
# using numpy function

start = time.time()
np.dot(A,B)
end = time.time()
print("Elapsed (sec) = %s" % (end - start))

Elapsed (sec) = 57.228665351867676
